In [ ]:
!pip install tensorflow

In [1]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
from itertools import islice, cycle
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
from tqdm import tqdm



In [3]:
interactions_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ИТМО/Recsys/5/interactions_processed_kion.csv')
users_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ИТМО/Recsys/5/users_processed_kion.csv')
items_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ИТМО/Recsys/5/items_processed_kion.csv')

In [4]:
users_df.head()

user_id        age        income sex  kids_flg
0   973171  age_25_34  income_60_90   M      True
1   962099  age_18_24  income_20_40   M     False
2  1047345  age_45_54  income_40_60   F     False
3   721985  age_45_54  income_20_40   F     False
4   704055  age_35_44  income_60_90   F     False

In [5]:
items_df = items_df.rename(columns = {'id' : 'item_id'})

In [6]:
items_df.head()

item_id content_type                 title      title_orig  \
0    10711         film        поговори с ней  Hable con ella   
1     2508         film           голые перцы    Search Party   
2    10716         film      тактическая сила  Tactical Force   
3     7868         film                45 лет        45 Years   
4    16268         film  все решает мгновение            None   

                                             genres       countries  for_kids  \
0           драмы, зарубежные, детективы, мелодрамы         испания     False   
1                  зарубежные, приключения, комедии             сша     False   
2  криминал, зарубежные, триллеры, боевики, комедии          канада     False   
3                      драмы, зарубежные, мелодрамы  великобритания     False   
4                драмы, спорт, советские, мелодрамы            ссср     False   

   age_rating   studios         directors  \
0        16.0   unknown  педро альмодовар   
1        16.0   unknown    скот армстронг   
2        16.0   unknown  адам п. калтраро   
3        16.0   unknown         эндрю хэй   
4        12.0  ленфильм  виктор садовский   

                                              actors  \
0  Адольфо Фернандес, Ана Фернандес, Дарио Гранди...   
1  Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...   
2  Адриан Холмс, Даррен Шалави, Джерри Вассерман,...   
3  Александра Риддлстон-Барретт, Джеральдин Джейм...   
4  Александр Абдулов, Александр Демьяненко, Алекс...   

                                         description  \
0  Мелодрама легендарного Педро Альмодовара «Пого...   
1  Уморительная современная комедия на популярную...   
2  Профессиональный рестлер Стив Остин («Все или ...   
3  Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...   
4  Расчетливая чаровница из советского кинохита «...   

                                            keywords release_year_cat  
0  Поговори, ней, 2002, Испания, друзья, любовь, ...        2000-2010  
1  Голые, перцы, 2014, США, друзья, свадьбы, прео...        2010-2020  
2  Тактическая, сила, 2011, Канада, бандиты, ганг...        2010-2020  
3  45, лет, 2015, Великобритания, брак, жизнь, лю...        2010-2020  
4  Все, решает, мгновение, 1978, СССР, сильные, ж...        1970-1980

In [7]:
interactions_df.head()

user_id  item_id last_watch_dt  total_dur  watched_pct
0   176549     9506    2021-05-11       4250           72
1   699317     1659    2021-05-29       8317          100
2   656683     7107    2021-05-09         10            0
3   864613     7638    2021-07-05      14483          100
4   964868     9506    2021-04-30       6725          100

In [8]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()


user_id  age_age_18_24  age_age_25_34  age_age_35_44  age_age_45_54  \
0   973171              0              1              0              0   
1   962099              1              0              0              0   
2  1047345              0              0              0              1   
3   721985              0              0              0              1   
4   704055              0              0              1              0   

   age_age_55_64  age_age_65_inf  age_age_unknown  income_income_0_20  \
0              0               0                0                   0   
1              0               0                0                   0   
2              0               0                0                   0   
3              0               0                0                   0   
4              0               0                0                   0   

   income_income_150_inf  income_income_20_40  income_income_40_60  \
0                      0                    0                    0   
1                      0                    1                    0   
2                      0                    0                    1   
3                      0                    1                    0   
4                      0                    0                    0   

   income_income_60_90  income_income_90_150  income_income_unknown  sex_F  \
0                    1                     0                      0      0   
1                    0                     0                      0      0   
2                    0                     0                      0      1   
3                    0                     0                      0      1   
4                    1                     0                      0      1   

   sex_M  sex_sex_unknown  kids_flg_False  kids_flg_True  
0      1                0               0              1  
1      1                0               1              0  
2      0                0               1              0  
3      0                0               1              0  
4      0                0               1              0

In [9]:
items_df.head()

item_id content_type                 title      title_orig  \
0    10711         film        поговори с ней  Hable con ella   
1     2508         film           голые перцы    Search Party   
2    10716         film      тактическая сила  Tactical Force   
3     7868         film                45 лет        45 Years   
4    16268         film  все решает мгновение            None   

                                             genres       countries  for_kids  \
0           драмы, зарубежные, детективы, мелодрамы         испания     False   
1                  зарубежные, приключения, комедии             сша     False   
2  криминал, зарубежные, триллеры, боевики, комедии          канада     False   
3                      драмы, зарубежные, мелодрамы  великобритания     False   
4                драмы, спорт, советские, мелодрамы            ссср     False   

   age_rating   studios         directors  \
0        16.0   unknown  педро альмодовар   
1        16.0   unknown    скот армстронг   
2        16.0   unknown  адам п. калтраро   
3        16.0   unknown         эндрю хэй   
4        12.0  ленфильм  виктор садовский   

                                              actors  \
0  Адольфо Фернандес, Ана Фернандес, Дарио Гранди...   
1  Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...   
2  Адриан Холмс, Даррен Шалави, Джерри Вассерман,...   
3  Александра Риддлстон-Барретт, Джеральдин Джейм...   
4  Александр Абдулов, Александр Демьяненко, Алекс...   

                                         description  \
0  Мелодрама легендарного Педро Альмодовара «Пого...   
1  Уморительная современная комедия на популярную...   
2  Профессиональный рестлер Стив Остин («Все или ...   
3  Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...   
4  Расчетливая чаровница из советского кинохита «...   

                                            keywords release_year_cat  
0  Поговори, ней, 2002, Испания, друзья, любовь, ...        2000-2010  
1  Голые, перцы, 2014, США, друзья, свадьбы, прео...        2010-2020  
2  Тактическая, сила, 2011, Канада, бандиты, ганг...        2010-2020  
3  45, лет, 2015, Великобритания, брак, жизнь, лю...        2010-2020  
4  Все, решает, мгновение, 1978, СССР, сильные, ж...        1970-1980

In [10]:
item_cat_feats = ['content_type', 'release_year_cat',
                  'for_kids', 'age_rating',
                  'studios', 'countries', 'directors']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1)

items_ohe_df.head()

item_id  content_type_film  content_type_series  \
0    10711                  1                    0   
1     2508                  1                    0   
2    10716                  1                    0   
3     7868                  1                    0   
4    16268                  1                    0   

   release_year_cat_1920-1930  release_year_cat_1930-1940  \
0                           0                           0   
1                           0                           0   
2                           0                           0   
3                           0                           0   
4                           0                           0   

   release_year_cat_1940-1950  release_year_cat_1950-1960  \
0                           0                           0   
1                           0                           0   
2                           0                           0   
3                           0                           0   
4                           0                           0   

   release_year_cat_1960-1970  release_year_cat_1970-1980  \
0                           0                           0   
1                           0                           0   
2                           0                           0   
3                           0                           0   
4                           0                           1   

   release_year_cat_1980-1990  ...  directors_ярив хоровиц  \
0                           0  ...                       0   
1                           0  ...                       0   
2                           0  ...                       0   
3                           0  ...                       0   
4                           0  ...                       0   

   directors_ярон зильберман  directors_ярополк лапшин  \
0                          0                         0   
1                          0                         0   
2                          0                         0   
3                          0                         0   
4                          0                         0   

   directors_ярослав лупий  directors_ярроу чейни, скотт моужер  \
0                        0                                    0   
1                        0                                    0   
2                        0                                    0   
3                        0                                    0   
4                        0                                    0   

   directors_ясина сезар  directors_ясуоми умэцу  \
0                      0                       0   
1                      0                       0   
2                      0                       0   
3                      0                       0   
4                      0                       0   

   directors_ёдзи фукуяма, ацуко фукусима, николас де креси, синъитиро ватанабэ, сёдзи кавамори  \
0                                                  0                                              
1                                                  0                                              
2                                                  0                                              
3                                                  0                                              
4                                                  0                                              

   directors_ёлкин туйчиев  directors_ён сан-хо  
0                        0                    0  
1                        0                    0  
2                        0                    0  
3                        0                    0  
4                        0                    0  

[5 rows x 8589 columns]

In [22]:
vectorizer = TfidfVectorizer()
genres_tfidf = vectorizer.fit_transform(items_df['genres'].astype(str))
genres_tfidf_df = pd.DataFrame(genres_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
items_ohe_df = pd.concat([items_ohe_df, genres_tfidf_df], axis=1)
items_ohe_df

item_id  content_type_film  content_type_series  \
8         9853                  1                    0   
10        3526                  1                    0   
15       15076                  1                    0   
16        2904                  1                    0   
20        2635                  1                    0   
...        ...                ...                  ...   
15957    15610                  0                    1   
15958     6443                  0                    1   
15959     2367                  0                    1   
15960    10632                  0                    1   
15961     4538                  0                    1   

       release_year_cat_1920-1930  release_year_cat_1930-1940  \
8                               0                           0   
10                              0                           0   
15                              0                           0   
16                              0                           0   
20                              0                           0   
...                           ...                         ...   
15957                           0                           0   
15958                           0                           0   
15959                           0                           0   
15960                           0                           0   
15961                           0                           0   

       release_year_cat_1940-1950  release_year_cat_1950-1960  \
8                               0                           0   
10                              0                           0   
15                              0                           0   
16                              0                           0   
20                              0                           0   
...                           ...                         ...   
15957                           0                           0   
15958                           0                           0   
15959                           0                           0   
15960                           0                           0   
15961                           0                           0   

       release_year_cat_1960-1970  release_year_cat_1970-1980  \
8                               0                           0   
10                              0                           0   
15                              0                           0   
16                              0                           0   
20                              0                           0   
...                           ...                         ...   
15957                           0                           0   
15958                           0                           0   
15959                           0                           0   
15960                           0                           0   
15961                           0                           0   

       release_year_cat_1980-1990  ...  directors_ярив хоровиц  \
8                               0  ...                       0   
10                              0  ...                       0   
15                              0  ...                       0   
16                              0  ...                       0   
20                              0  ...                       0   
...                           ...  ...                     ...   
15957                           0  ...                       0   
15958                           0  ...                       0   
15959                           0  ...                       0   
15960                           0  ...                       0   
15961                           0  ...                       0   

       directors_ярон зильберман  directors_ярополк лапшин  \
8                              0                         0   
10                             0                  

In [23]:
interactions_df.item_id.value_counts()

9728     23999
13865    21766
15297    19691
10440    19201
3734     16744
         ...  
2401         1
12787        1
14478        1
11375        1
5304         1
Name: item_id, Length: 6897, dtype: int64

In [25]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

interactions_df = interactions_df[interactions_df.watched_pct > 10]

valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 65974
N items before: 6897

N users after: 65552
N items after: 5303


In [26]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65552
5303


In [27]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


user_id  item_id last_watch_dt  total_dur  watched_pct    uid   iid
0    176549     9506    2021-05-11       4250           72  10561  3040
1    699317     1659    2021-05-29       8317          100  41858   503
6   1016458      354    2021-08-14       1672           25  60634   100
14     5324     8437    2021-04-18       6598           92    309  2685
18   927973     9617    2021-06-19       8422          100  55430  3065

In [28]:
interactions_vec = np.zeros((interactions_df.uid.nunique(),
                             interactions_df.iid.nunique()))

for user_id, item_id in zip(interactions_df.uid, interactions_df.iid):
    interactions_vec[user_id, item_id] += 1


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [29]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

print(set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique()))

5303
5303
65552
65552
set()


In [30]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

In [31]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

In [32]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)

    return loss

In [33]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            uid_i = randint(0, interactions.shape[0]-1)
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            neg_i = np.random.choice(range(interactions.shape[1]))
            uid_meta.append(users.iloc[uid_i])
            uid_interaction.append(interactions_vec[uid_i])
            pos.append(items.iloc[pos_i])
            neg.append(items.iloc[neg_i])
        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]


In [34]:
def weighted_generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            uid_i = np.random.choice(range(interactions.shape[0]), p=np.sum(interactions, axis=1) / np.sum(interactions))
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i] / np.sum(interactions[uid_i]))
            neg_i = np.random.choice(range(interactions.shape[1]), p=(1 - interactions[uid_i]) / np.sum(1 - interactions[uid_i]))
            uid_meta.append(users.iloc[uid_i])
            uid_interaction.append(interactions_vec[uid_i])
            pos.append(items.iloc[pos_i])
            neg.append(items.iloc[neg_i])

        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]


In [35]:
gen = weighted_generator(items=items_ohe_df.drop(["item_id"], axis=1),
                users=users_ohe_df.drop(["user_id"], axis=1),
                interactions=interactions_vec)

ret = next(gen)

print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 5303)
вектор 'хорошего' айтема: (1024, 8588)
вектор 'плохого' айтема: (1024, 8588)

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 5303)


In [36]:
N_FACTORS = 128

ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1], )
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (8588,)
USER_META_MODEL_SHAPE: (19,)
USER_INTERACTION_MODEL_SHAPE: (5303,)


In [37]:
def item_model(n_factors=N_FACTORS):
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                               kernel_regularizer=keras.regularizers.l2(1e-6),
                               activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)
    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)

    add = keras.layers.Add()([layer_1, layer_2])
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)

    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)
    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_meta)
    layer_1_interaction = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_interaction)
    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1_meta)

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])

    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])

    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(concat_meta_interaction)

    return keras.models.Model([inp_meta, inp_interaction], out)

i2v = item_model()
u2v = user_model()

ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

ancor = u2v([ancor_meta_in, ancor_interaction_in])
pos = i2v(pos_in)
neg = i2v(neg_in)

res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [38]:
model_name = 'recsys_resnet_linear'

t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

check = keras.callbacks.ModelCheckpoint(filepath=model_name + '/epoch{epoch}-{loss:.2f}.h5', monitor="loss")


In [39]:
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [40]:

item_model().summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 8588)]               0         []                            
                                                                                                  
 dense_7 (Dense)             (None, 128)                  1099264   ['input_8[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 128)                  16384     ['dense_7[0][0]']             
                                                                                                  
 add_2 (Add)                 (None, 128)                  0         ['dense_7[0][0]',             
                                                                     'dense_8[0][0]']       

In [41]:
user_model().summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 dense_10 (Dense)            (None, 128)                  2432      ['input_9[0][0]']             
                                                                                                  
 dense_12 (Dense)            (None, 128)                  16384     ['dense_10[0][0]']            
                                                                                                  
 input_10 (InputLayer)       [(None, 5303)]               0         []                            
                                                                                            

In [42]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 5303)]               0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 8588)]               0         []                            
                                                                                                  
 input_7 (InputLayer)        [(None, 8588)]               0         []                            
                                                                                            

In [43]:
model.fit(generator(items=items_ohe_df.drop(["item_id"], axis=1),
                    users=users_ohe_df.drop(["user_id"], axis=1),
                    interactions=interactions_vec,
                    batch_size=64),
          steps_per_epoch=100,
          epochs=30,
          initial_epoch=0,
          callbacks=[decay, t_board, check]
)

Epoch 1/30
100/100 [==============================] - 20s 185ms/step - loss: 0.3040 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 20s 196ms/step - loss: 0.2332 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 18s 181ms/step - loss: 0.2122 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 18s 180ms/step - loss: 0.1996 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 18s 179ms/step - loss: 0.1919 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 18s 183ms/step - loss: 0.1862 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 18s 180ms/step - loss: 0.1789 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 19s 188ms/step - loss: 0.1768 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - 19s 191ms/step - loss: 0.1817 - lr: 0.0010
Epoch 10/30
100/100 [==============================] - 18s 180ms/step - loss: 0.1689 - lr: 0.0010
Epoch 11/30
100/100 [========

In [44]:
rand_uid = np.random.choice(list(users_ohe_df.index))

user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid]
user_interaction_vec = interactions_vec[rand_uid]

rand_iid = np.random.choice(list(items_ohe_df.index))
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid]

user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1),
                        np.array(user_interaction_vec).reshape(1, -1)])

item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

from sklearn.metrics.pairwise import euclidean_distances as ED

ED(user_vec, item_vec)

1/1 [==============================] - 0s 77ms/step


array([[1.6795185]], dtype=float32)

In [47]:
users_meta_feats = users_ohe_df.drop(["user_id"], axis=1)
users_interaction_vec = interactions_vec
print(users_meta_feats.shape)
print(users_interaction_vec.shape)

np.array(users_meta_feats).shape

(65552, 19)
(65552, 5303)


(65552, 19)

In [ ]:
users_vec = u2v.predict([np.array(users_meta_feats),
                        np.array(users_interaction_vec)])
print(users_vec.shape)
print(items_vecs.shape)

In [ ]:
dists = ED(users_vec, items_vecs)

print(dists.shape)

In [ ]:
top10_iids = np.argsort(dists, axis=1)[:,:10]

print(top10_iids.reshape(dists.shape[0], 10).shape)
print(top10_iids.shape)

top10_iids

In [ ]:
top10_iids_item = [iid_to_item_id[iid] for iid in top10_iids.reshape(-1)]
top10_iids_item = np.array(top10_iids_item).reshape(top10_iids.shape)
print(top10_iids_item.shape)

In [ ]:
df_dssm = pd.DataFrame(columns = ['user_id', 'item_id'])
df_dssm = pd.DataFrame({'user_id': [uid_to_user_id[uid] for uid in np.arange(top10_iids_item.shape[0])]})
df_dssm['item_id'] = list(top10_iids_item)
df_dssm = df_dssm.explode('item_id')
df_dssm['rank'] = df_dssm.groupby('user_id').cumcount() + 1
df_dssm = df_dssm.groupby('user_id').agg({'item_id': list}).reset_index()

df_dssm

In [ ]:
df_dssm.to_csv('dssm_recos.csv')